In [6]:
# load entities
import json

with open(data_path+'entities.json', 'r') as fin:
    entities = json.load(fin)

corpus = []
for _id, label in entities.items():
    corpus.append({"_id": _id, "title": "", "text": label, "metadata": {}})

print(len(corpus), 'entities')
print(corpus[0])

28497 entities
{'_id': 'Q1938494', 'title': '', 'text': 'Mirosław Bork', 'metadata': {}}


In [15]:
data_path = '/ivi/ilps/personal/svakule/spoken_qa/'

path_to_questions = data_path + 'sqs_valid_wikidata2018_09_11.txt'

with open(path_to_questions) as fin:
    lines = fin.readlines()
    print(len(lines), 'questions in total')

queries, qrels = [], []
indices = []
for i, l in enumerate(lines):
    s, p, o, q = lines[i].strip('\n').split('\t')
    if s in entities:
        indices.append(i)
        queries.append({"_id": str(i), "text": q, "metadata": {}})
        qrels.append([str(i), s, '1'])

print(len(queries), 'samples with entities recognised')
print(queries[0], qrels[0])

1026 questions in total
995 samples with entities recognised
{'_id': '0', 'text': 'where was sasha vujačić born', 'metadata': {}} ['0', 'Q318926', '1']


In [16]:
# save dataset
import jsonlines

dataset = 'WD18/'

corpus_path = data_path + dataset + "entities.jsonl"
query_path = data_path + dataset + "text_questions.jsonl"
qrels_path = data_path + dataset + "dev.tsv"


with open(corpus_path, 'w') as out_file:
    for d in corpus:
        out_file.write(json.dumps(d))
        out_file.write("\n")
        
with open(query_path, 'w') as out_file:
    for d in queries:
        out_file.write(json.dumps(d))
        out_file.write("\n")

with open(qrels_path, 'w') as out_file:
    for qrel in qrels:
        out_file.write('\t'.join(qrel)+'\n')

In [17]:
from beir.datasets.data_loader import GenericDataLoader

corpus, queries, qrels = GenericDataLoader(
    corpus_file=corpus_path, 
    query_file=query_path, 
    qrels_file=qrels_path).load_custom()

# ASR Transcriptions

In [25]:
# load queries and qrels
import os
import torch
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
from datasets import load_dataset
import soundfile as sf

model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")
tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")

wav_path = "/ivi/ilps/personal/svakule/spoken_qa/sqs_valid_wikidata2018_09_11/"

tqueries = {}
for file in os.listdir(wav_path):
    i = int(file.split('.')[0]) - 1
    if i in indices:
        speech, samplerate = sf.read(wav_path+file)

        input_values = tokenizer(speech, return_tensors="pt", padding="longest").input_values    
        logits = model(input_values).logits
        predicted_ids = torch.argmax(logits, dim=-1)
        transcription = tokenizer.batch_decode(predicted_ids)[0].lower()
        
        q_id = 'q%d' % i
        tqueries[q_id] = transcription

print(len(tqueries), 'questions transcribed')

Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


218.wav
679.wav
391.wav
436.wav
844.wav
340.wav
895.wav
721.wav
264.wav
605.wav
173.wav
960.wav
512.wav
57.wav
838.wav
86.wav
263.wav
602.wav
174.wav
515.wav
967.wav
50.wav
81.wav
108.wav
569.wav
843.wav
431.wav
396.wav
892.wav
347.wav
726.wav
969.wav
785.wav
831.wav
443.wav
335.wav
492.wav
1013.wav
754.wav
211.wav
670.wav
106.wav
567.wav
915.wav
22.wav
398.wav
349.wav
728.wav
216.wav
677.wav
101.wav
912.wav
560.wav
25.wav
438.wav
782.wav
444.wav
836.wav
59.wav
88.wav
495.wav
332.wav
1014.wav
753.wav
927.wav
555.wav
134.wav
693.wav
642.wav
584.wav
223.wav
10.wav
529.wav
148.wav
199.wav
1021.wav
766.wav
307.wav
471.wav
803.wav
289.wav
639.wav
258.wav
1026.wav
761.wav
300.wav
804.wav
476.wav
552.wav
920.wav
694.wav
133.wav
645.wav
224.wav
583.wav
17.wav
878.wav
520.wav
952.wav
287.wav
141.wav
190.wav
637.wav
983.wav
256.wav
65.wav
478.wav
713.wav
372.wav
876.wav
404.wav
19.wav
929.wav
714.wav
375.wav
403.wav
871.wav
280.wav
955.wav
527.wav
146.wav
630.wav
197.wav
251.wav
984.wav
768.wav


In [24]:
print(transcriptions[0])

what is marfines van scok vix job
